# Convert TSV Data To Parquet with Athena

In this notebook, we will show you how you can easily convert that data now into Apache Parquet file format.

<img src="img/athena_convert_parquet.png" width="60%" align="left">

In [1]:
import boto3
import sagemaker

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

# Install PyAthena

In [2]:
!pip install -q PyAthena==1.10.7

In [3]:
from pyathena import connect
from pyathena.pandas_cursor import PandasCursor
from pyathena.util import as_pandas

# Create Parquet Files from TSV Table

As you can see from the query below, we’re also adding a new `year` column to our dataset by converting the `review_date` string to a date format, and then cast the year out of the date. Let’s store the year value as an integer. And let's partition the Parquet data by `Product Category`.

In [4]:
# Set S3 path to Parquet data
s3_path_parquet = 's3://{}/amazon-reviews-pds/parquet'.format(bucket)

# Set Athena parameters
database_name = 'dsoaws'
table_name_tsv = 'amazon_reviews_tsv'
table_name_parquet = 'amazon_reviews_parquet'

In [5]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = 's3://{0}/athena/staging'.format(bucket)

In [6]:
# SQL statement to execute
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (format = 'PARQUET', external_location = '{}', partitioned_by = ARRAY['product_category']) AS
SELECT marketplace,
         customer_id,
         review_id,
         product_id,
         product_parent,
         product_title,
         star_rating,
         helpful_votes,
         total_votes,
         vine,
         verified_purchase,
         review_headline,
         review_body,
         CAST(YEAR(DATE(review_date)) AS INTEGER) AS year,
         DATE(review_date) AS review_date,
         product_category
FROM {}.{}""".format(database_name, table_name_parquet, s3_path_parquet, database_name, table_name_tsv)

print(statement)

CREATE TABLE IF NOT EXISTS dsoaws.amazon_reviews_parquet
WITH (format = 'PARQUET', external_location = 's3://sagemaker-us-west-2-250107111215/amazon-reviews-pds/parquet', partitioned_by = ARRAY['product_category']) AS
SELECT marketplace,
         customer_id,
         review_id,
         product_id,
         product_parent,
         product_title,
         star_rating,
         helpful_votes,
         total_votes,
         vine,
         verified_purchase,
         review_headline,
         review_body,
         CAST(YEAR(DATE(review_date)) AS INTEGER) AS year,
         DATE(review_date) AS review_date,
         product_category
FROM dsoaws.amazon_reviews_tsv


# Execute statement using connection cursor
This can take a few minutes.  Please be patient.

In [7]:
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

# Load partitions by running `MSCK REPAIR TABLE`

As a last step, we need to load the Parquet partitions. To do so, just issue the following SQL command: 

In [8]:
statement = 'MSCK REPAIR TABLE {}.{}'.format(database_name, table_name_parquet)

print(statement)

MSCK REPAIR TABLE dsoaws.amazon_reviews_parquet


In [9]:
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

# Show the Partitions

In [10]:
statement = 'SHOW PARTITIONS {}.{}'.format(database_name, table_name_parquet)

print(statement)

SHOW PARTITIONS dsoaws.amazon_reviews_parquet


In [11]:
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

df_partitions = as_pandas(cursor)
df_partitions.head(5)

,partition
0,product_category=Digital_Software
1,product_category=Digital_Video_Games


# Run Sample Query

In [12]:
product_category = 'Digital_Software'

statement = """SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100""".format(database_name, table_name_parquet, product_category)

print(statement)

SELECT * FROM dsoaws.amazon_reviews_parquet
    WHERE product_category = 'Digital_Software' LIMIT 100


In [13]:
# Execute statement using connection cursor
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

df = as_pandas(cursor)
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,year,review_date,product_category
0,US,20704715,R2AIUWIU5QQLHO,B00FFINOWS,875090538,"TurboTax Deluxe Fed, Efile and State 2013",5,0,0,N,Y,Much better than before,This is the best version of TurboTax yet. I fo...,2014,2014-03-13,Digital_Software
1,US,28236644,R181XLBQ79SRKW,B00E7X9RUK,109813655,Quicken Deluxe 20,5,0,1,N,Y,This program is exactly what I was looking for!,This program is amazing! I knew my budget woul...,2014,2014-03-13,Digital_Software
2,US,9916801,R6DZGFO4SWOF7,B00FFIMBOA,331894956,TurboTax Basic Fed and Efile 2013,2,1,4,N,Y,Not Happy,I was happy with the program it self but the f...,2014,2014-03-13,Digital_Software
3,US,29694422,R3T1BCHXHPU5C2,B00G0DXA9Y,595906078,H&R Block At Home 2013 Deluxe + State,5,0,0,N,Y,So easy to use!,This is the 4th or 5th year in a row I've used...,2014,2014-03-13,Digital_Software
4,US,21225441,R4DH5EY1TS32W,B00FFINRG6,805918609,"TurboTax Deluxe Fed, Efile and State 2013",5,0,0,N,Y,Easy to use,Consistently easy to use. I have used for clos...,2014,2014-03-13,Digital_Software


In [14]:
if not df.empty:
    print('Congratulations!  Your data is has been converted to Parquet and you are ready to proceed.')
else:
    print('PLEASE RE-RUN THIS NOTEBOOK AS YOUR DATA HAS NOT BEEN CONVERTED TO PARQUET PROPERLY!!')

Congratulations!  Your data is has been converted to Parquet and you are ready to proceed.


# Done! 

In just a few steps we have set up Amazon Athena to connect to our Amazon Customer Reviews TSV files, and transformed them into Apache Parquet file format. 

You might have noticed that our second sample query finished in a fraction of the time compared to the one before we ran on the TSV table. We sped up our query results by leveraging our data being stored as Parquet and partitioned by `product_category`. 


In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();